# Video Reconstruction for Volleyball dataset
**Author:** Christian Byron  **Date:** 26-Mar-21

This notebook provides an understanding of the Volleyball dataset by reconstructing it into a video gui. The volleyball dataset (available [here](https://github.com/mostafa-saad/deep-activity-rec#dataset)) was created by Ibrahim et al. <sub>[1]</sub> using publicly available YouTube volleyball videos.

- [x] Figure out how to change GUI to select the video from the list when established form the subdirectories
- [x] Change the frames collection to be driven by selecting video ... for now we just provide all of the video frames
- [ ] Add next steps to read in the annotations 
- [ ] Some excepion handling on widgets when moving back to prior videos (upsets min and max values).. May mean redesigning the video data store again. 

### Step 1 - Find all videos and individual frames

- The dataset contains 4830 annotated frames that were handpicked from 55 videos with 9 player action labels and 8 team activity labels. 
- Each video has a directory for it with sequential IDs (0, 1...54)
- Inside each video directory, a set of directories corresponds to annotated frames (e.g. volleyball/39/29885)
- Each frame directory has 41 images (20 images before target frame, target frame, 20 frames after target frame)
- Each video directory has annotations.txt file that contains selected frames annotations.





In [4]:
import os
import sys

base_video_collection_path = \
    "C:\\Users\\s441606\\OneDrive - University of Canberra\\2021 Semester 1\\z- Learning Python\\Jupyter\\videos"

video_folder_list = os.listdir(base_video_collection_path)
videos = {}

for video_folder in video_folder_list:

    video_frames = list()
    video_starting_frame = sys.maxsize
    
    for subdir, dirs, files in os.walk(base_video_collection_path + "\\" + video_folder):
        for dir in dirs:
            video_starting_frame = min(int(dir), video_starting_frame)
        for file in files:
            if file.endswith(".jpg"):
                video_frames.append(os.path.join(subdir, file))
    
    video_obj = {"starting_frame":video_starting_frame, "frames":video_frames} 
    
    videos.update({video_folder:video_obj})



### Step 2 - Build the GUI Function to show video frames

In [7]:
import ipywidgets as widgets

imageArea = widgets.Image()

dropdown = widgets.Dropdown(
    options= video_folder_list,
    value ='1',
    description='Video #:',
    disabled=False,
)

play = widgets.Play(
        step=1,
        interval=50,
        description="Press play",
        disabled=False
)

slider = widgets.IntSlider()

widgets.jslink((play, 'value'), (slider, 'value'))

def on_dropdown_value_change(change): 
    video_selected = videos[change['new']]
    video_start = video_selected["starting_frame"]
    video_end = video_start + len(video_selected["frames"])

    slider.max = video_end
    play.max = video_end
    slider.min = video_start
    play.min = video_start


dropdown.observe(on_dropdown_value_change, names='value')

def on_value_change(change):
    video_selected = videos[dropdown.value]
    video_start = video_selected["starting_frame"]
    frame_index = change['new'] - video_start 
    
    file = open(f"{video_selected['frames'][frame_index]}","rb")
    imageArea.value = file.read()
 
slider.observe(on_value_change, names='value')
    

controls = widgets.VBox([widgets.HBox([dropdown, play, slider]),imageArea])

# trigger the initial video to correctly set the slider values and load the first image
dropdown.value = '0'
controls

#### Lessons Learnt
- A lot of learning about the iPython widgets (inc lack of documentation on class interfaces). Especially on how to use observe method to select and run the video(cascading value changes)
- How to show a image using ipyWidgets - may be useful later - or may be replaced with other image libraries (eg Matlablib)
- A bit of a challenge to create the complex store of videos in a dictionary


#### References
[1]  Mostafa S. Ibrahim et al. “A Hierarchical Deep Temporal Model for Group Activity Recognition”. In:2016 IeeeConference on Computer Vision and Pattern Recognition.  IEEE  Conference  on  Computer  Vision  and  PatternRecognition.  2016,  pp.  1971–1980.isbn:  978-1-4673-8851-1